## 동별 세대수 가져오기
+ 작성: 김지훈
+ 수정: 임경호

### 1. 지정한 년월에 사용 가능한 최신의 법정동, 행정동 코드 자료를 테이블에서 가져온다.

In [1]:
import pandas as pd
import requests
from tqdm.notebook import tqdm
import json
from datetime import datetime
import calendar

import sys
module_path = "D:\PythonProject\data-gatherer\common"
sys.path.append(module_path)
import dbconnect

In [2]:
#추출하고자 하는 년월
input_year = 2023
input_month = 7
# 년월: yyyymm 형식 문자열
month = str(input_year) + str(input_month).zfill(2)
# month = '202306'   
last_day = calendar.monthrange(input_year, input_month)[1]
# print(year, month, last_day)
month_end_date = datetime(input_year, input_month, last_day).strftime("%Y%m%d")

In [3]:
# 해당 년월에 사용가능한, 저장된 데이터의 마지막 일자 가져오기
conn = dbconnect.db_connect("DEMO_DM")
cur = conn.cursor()

query = "SELECT 업데이트일자 FROM city WHERE 업데이트일자 <= '" + month_end_date + "' ORDER BY 업데이트일자 DESC LIMIT 1"
cur.execute(query)
row = cur.fetchone()
stored_date = row[0]     

conn.close()

In [4]:
# 지역코드 테이블에서 데이터 읽어오기
engine = dbconnect.db_engine("DEMO_DM")
query = "SELECT * FROM city WHERE 업데이트일자 = '" + stored_date + "'"
result = pd.read_sql(query, engine)

In [5]:
df_code_table = result.copy()
df_code_table

,업데이트일자,행정동코드,시도명,시군구명,읍면동명,법정동코드,동리명
0,20230703,2700000000,대구광역시,,,2700000000,대구광역시
1,20230703,2711000000,대구광역시,중구,,2711000000,중구
2,20230703,2711051700,대구광역시,중구,동인동,2711010100,동인동1가
3,20230703,2711051700,대구광역시,중구,동인동,2711010200,동인동2가
4,20230703,2711051700,대구광역시,중구,동인동,2711010300,동인동3가
...,...,...,...,...,...,...,...
792,20230703,4785032000,경상북도,칠곡군,동명면,4785032034,남원리
793,20230703,4785032000,경상북도,칠곡군,동명면,4785032035,득명리
794,20230703,4785032000,경상북도,칠곡군,동명면,4785032036,기성리
795,20230703,4785032000,경상북도,칠곡군,동명면,4785032037,구덕리


### 2. 세대수 데이터 추출을 위한 법정동 코드 전처리

Open API를 사용하기 위해 **법정동 코드를 전처리**하는 과정입니다.  
- 해당 API는 법정동코드를 입력하였을 때, 값이 나오도록 되어 있습니다.  
- API사용 할 때, **구, 시, 리**의 법정동 코드를 제거하고 **동,읍,면** 법정동 코드만 남겨둬야합니다. 

법정동 코드는 총 10자리로 10자리의 구성은 아래와 같습니다.  
 - 시도(2) + 시군구(3) + 읍면동(3) + 리(2)

In [6]:
# 전제 코드 데이터
df_code_table_pre = df_code_table.copy()
df_code_table_pre.shape

(797, 7)

In [7]:
# '시군구' 단위 및 '읍면동' 단위의 데이터만 남겨놓는다
df_code_table_pre = df_code_table_pre[df_code_table_pre['법정동코드'].str.endswith('00')]
# '시군구' 단위 데이터 삭제
idx = df_code_table_pre[df_code_table_pre['법정동코드'].str.endswith('00000')].index
df_code_table_pre.drop(idx, inplace=True)
# 최종적으로 '읍면동' 코드만 남는다.
df_code_table_pre.shape

(352, 7)

### 각 지역별로 Data Set 구성

In [8]:
#구 별로 법정동코드를 추출하여 list형식으로 변환
def make_code_list_for_daegu(df,gu_name):
    df_gu= df[df['시군구명']==gu_name]
    list_gu=  df_gu['법정동코드']
    list_gu = list_gu.unique()
    return list_gu

In [9]:
# 각 도시별 법정동 코드 불러오기

#대구광역시
df_daegu_code = df_code_table_pre[df_code_table_pre['시도명'].str.contains('대구광역시')]
df_daegu_code= df_daegu_code.dropna(subset=['읍면동명']) #null값 제거
idx = df_daegu_code[df_daegu_code['법정동코드'].str[-4:] == "0000"].index #리 값 제거
df_daegu_code.drop(idx , inplace=True)

list_daegu_cd =  df_daegu_code['법정동코드']
list_daegu_cd = list_daegu_cd.unique() #대구광역시의 법정동코드

#중간에 오류가 날 수 있어 구별로 나누어 작업 진행
dict_region_codes = {}  # 딕셔너리
dict_region_codes["남구"] =  make_code_list_for_daegu(df_daegu_code,"남구")
dict_region_codes["달서구"] =  make_code_list_for_daegu(df_daegu_code,"달서구")
dict_region_codes["달성군"] =  make_code_list_for_daegu(df_daegu_code,"달성군")
dict_region_codes["동구"] =  make_code_list_for_daegu(df_daegu_code,"동구")
dict_region_codes["북구"] =  make_code_list_for_daegu(df_daegu_code,"북구")
dict_region_codes["서구"] =  make_code_list_for_daegu(df_daegu_code,"서구")
dict_region_codes["수성구"] =  make_code_list_for_daegu(df_daegu_code,"수성구")
dict_region_codes["중구"] =  make_code_list_for_daegu(df_daegu_code,"중구")

#경산시
df_gyeongsan_code= df_code_table_pre[df_code_table_pre['시군구명']=='경산시'] 
df_gyeongsan_code= df_gyeongsan_code.dropna(subset=['읍면동명'])
list_gyeongsan_cd =  df_gyeongsan_code['법정동코드']
dict_region_codes["경산시"] = list_gyeongsan_cd.unique()#유일한 값만 남겨둠

#고령군
df_goryeong_code= df_code_table_pre[df_code_table_pre['시군구명']=='고령군']
df_goryeong_code= df_goryeong_code.dropna(subset=['읍면동명'])
list_goryeong_cd =  df_goryeong_code['법정동코드']
dict_region_codes["고령군"] = list_goryeong_cd.unique()#유일한 값만 남겨둠

#칠곡군 동명면
df_dongmyeong_code= df_code_table_pre[df_code_table_pre['읍면동명']=='동명면']
list_dongmyeong_cd =  df_dongmyeong_code['법정동코드']
dict_region_codes["칠곡군"] = list_dongmyeong_cd.unique()#유일한 값만 남겨둠


In [11]:
# 각 지역별로 지역코드(동별) 개수 확인
for key in dict_region_codes:
    print(f'{key}\t{len(dict_region_codes[key])}')

남구	3
달서구	24
달성군	9
동구	45
북구	31
서구	9
수성구	26
중구	57
경산시	36
고령군	8
칠곡군	1


### 3. Open API를 이용한 세대수 데이터 추출

In [12]:
import warnings
from time import sleep
warnings.filterwarnings(action='ignore')

In [13]:
service_url = 'http://apis.data.go.kr/1741000/stdgPpltnHhStus'
# 일반 인증키(Encoding)	
api_key = 'IVgu%2FZBjA6hpLryyEOpySC2RhogOhaJIUqlXN8Uyj3Gxw4s3dX0qMxfgXMTLl60%2Fs2EYAMUsyyzTqwVOnjoIhg%3D%3D'
level = '4'     # 조회결과구분. 광역시도 단위 : 1, 시군구 단위 : 2, 읍면동 단위 : 3, 읍면동 통반 단위 : 4(기본값 : 4)
reg_code = '1'  # 등록구분. 전체:1, 거주자:2, 거주불명자:3, 재외국민:4(기본값 : 1)
req_type = 'json'   # 타입. XML, JSON(기본값 : XML)
pageRows = '8000'    # 페이지 크기. 페이지당 목록 수(1~100)(기본값 : 10)
pageNo = '1'        # 페이지 번호. 기본값 : 1

In [14]:
#법정동 코드를 이용한 행정동/법정동별 세대수를 API로 구하는 함수
def make_hh_data(df,list_cd,month):
    for i in tqdm(list_cd): 
        url = f'{service_url}/selectStdgPpltnHhStus?serviceKey={api_key}&stdgCd={i}&srchFrYm={month}&srchToYm={month}&lv={level}&regSeCd={reg_code}&type={req_type}&numOfRows={pageRows}&pageNo={pageNo}'
        # url = 'https://apis.data.go.kr/1741000/stdgPpltnHhStus/selectStdgPpltnHhStus?serviceKey=IVgu%2FZBjA6hpLryyEOpySC2RhogOhaJIUqlXN8Uyj3Gxw4s3dX0qMxfgXMTLl60%2Fs2EYAMUsyyzTqwVOnjoIhg%3D%3D&stdgCd='+i+'&srchFrYm='+month+'&srchToYm='+month+'&lv=4&regSeCd=1&type=json&numOfRows=8000&pageNo=1'
        response = requests.get(url,verify=False)
        #데이터 값 출력해보기
        contents = response.text
        json_ob = json.loads(contents)
        body = json_ob['Response']['items']['item']
        dataframe = pd.json_normalize(body)
        df = pd.concat([df,dataframe],axis = 0) #데이터 결합
    return df

#시군구별 세대수를 알아보는 함수 >> 시군구 자료와 데이터 비교
def know_hhcnt(df):
    df['hhCnt'] =df['hhCnt'].astype(str).astype(int)
    df['femlNmprCnt'] =df['femlNmprCnt'].astype(str).astype(int)
    df['totNmprCnt'] =df['totNmprCnt'].astype(str).astype(int)
    df['maleNmprCnt'] =df['maleNmprCnt'].astype(str).astype(int)
    know_hhcnt = df.groupby('sggNm').sum()
    return know_hhcnt

#기존의 표와 같이 보기 편하게 순서 변경 및 컬럼 변경
def name_sort_change(df):
    #이름 변경
    df.rename(columns={'hhCnt':'세대수','tong':'통','femlNmprCnt':'여자인구수','stdgCd':'법정동코드','maleFemlRate':'남여비율',
                       'stdgNm':'법정동명','ban':'반','totNmprCnt':'총인구수','ctpvNm':'시도명','maleNmprCnt':'남자인구수',
                       'sggNm':'시군구명','dongNm':'행정동명','hhNmpr':'세대당인구','admmCd':'행정기관코드',
                       'statsYm':'통계년월','liNm':'리명'},inplace=True)
    #기존의 표와 같이 순서 변경
    df = df[['통계년월','법정동코드','시도명','시군구명','법정동명','리명','행정기관코드','행정동명','통',
             '반','총인구수','세대수','세대당인구','남자인구수','여자인구수','남여비율']]
    df['업데이트일자'] = datetime.now().strftime("%Y%m%d")
    return df

In [15]:
# 각 지역별로 데이터 수집 결과 초기화
dict_region_results = {}
for key in dict_region_codes:
    dict_region_results[key] = 0
print(dict_region_results)

{'남구': 0, '달서구': 0, '달성군': 0, '동구': 0, '북구': 0, '서구': 0, '수성구': 0, '중구': 0, '경산시': 0, '고령군': 0, '칠곡군': 0}


In [16]:
# 각 지역별로 API 호출하여 동별 세대수 데이터 가져오기
dict_region_data = {}
for key in dict_region_results:
    # 최대 5회 시도
    num_retry = 5
    for retry in range(num_retry):
        try:
            print(key, " try ", retry)
            df_region_cd_list = dict_region_codes[key]
            df_region_data = pd.DataFrame()
            df_region_data = make_hh_data(df_region_data, df_region_cd_list, month) #남구
            dict_region_data[key] = df_region_data
            dict_region_results[key] = 1    # 성공
            break
        except Exception:
            continue
    sleep(5)
    
# 각 지역별로 데이터 수집 결과 확인: 성공(1), 실패(0)
print("각 지역별 데이터 수집 결과")
for key in dict_region_results:
    print(f'{key}\t{dict_region_results[key]}')

남구  try  0


  0%|          | 0/3 [00:00<?, ?it/s]

달서구  try  0


  0%|          | 0/24 [00:00<?, ?it/s]

달성군  try  0


...

고령군  try  0


  0%|          | 0/8 [00:00<?, ?it/s]

칠곡군  try  0


  0%|          | 0/1 [00:00<?, ?it/s]

각 지역별 데이터 수집 결과
남구	1
달서구	1
달성군	1
동구	1
북구	1
서구	1
수성구	1
중구	1
경산시	1
고령군	1
칠곡군	1


In [17]:
df_house_hold = pd.DataFrame()
for key in dict_region_data:
    df_house_hold = pd.concat([df_house_hold, dict_region_data[key]], axis = 0)
df_house_hold.shape

(27655, 16)

In [18]:
df_house_hold

,hhCnt,tong,femlNmprCnt,stdgCd,maleFemlRate,stdgNm,ban,totNmprCnt,ctpvNm,maleNmprCnt,sggNm,dongNm,hhNmpr,admmCd,statsYm,liNm
0,9,1,6,2720010100,0.83,이천동,1,11,대구광역시,5,남구,이천동,1.22,2720051500,202307,NaN
1,25,1,24,2720010100,0.96,이천동,2,47,대구광역시,23,남구,이천동,1.88,2720051500,202307,NaN
2,35,1,26,2720010100,1.12,이천동,4,55,대구광역시,29,남구,이천동,1.57,2720051500,202307,NaN
3,52,1,35,2720010100,1.03,이천동,5,71,대구광역시,36,남구,이천동,1.37,2720051500,202307,NaN
4,55,2,69,2720010100,0.88,이천동,1,130,대구광역시,61,남구,이천동,2.36,2720051500,202307,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,18,17,18,4785032038,1.11,동명면,3,38,경상북도,20,칠곡군,동명면,2.11,4785032000,202307,봉암리
64,27,17,17,4785032038,1.53,동명면,4,43,경상북도,26,칠곡군,동명면,1.59,4785032000,202307,봉암리
65,12,17,10,4785032038,1.30,동명면,5,23,경상북도,13,칠곡군,동명면,1.92,4785032000,202307,봉암리
66,71,17,48,4785032038,1.50,동명면,6,120,경상북도,72,칠곡군,동명면,1.69,4785032000,202307,봉암리


In [19]:
# df_house_hold.drop_duplicates(keep = 'first',inplace=True) #중복값 제거 >> 중복값이 있을 때 첫번째 값만 남겨둠

In [20]:
print(know_hhcnt(df_house_hold))

        hhCnt                                               tong  femlNmprCnt  \
sggNm                                                                           
경산시    129170  1234566789101112133536141515373737373737373838...       132810   
고령군     16919  1111111112222222223262645678889101112131415161...        14689   
남구      76471  1111222223333344444455555566666667777778888888...        73516   
달서구    236819  1111112222223333334444455555566666677777788888...       271465   
달성군    114071  1111111111112222222222233333333344444444555555...       129545   
동구     161108  1111111111111111111111111111111222222222222222...       173627   
북구     192659  1191919191920202020202021212121211111122222233...       214392   
서구      81747  1111111111111111111112222222222222222222222333...        80236   
수성구    171298  1111111111111111111111111111122222222222222222...       211164   
중구      43214  1111122333334455554455162323232323232366666677...        44410   
칠곡군      2963  1111112222233

In [21]:
#데이터 컬럼명 변경 및 위치 변경
df_house_hold = name_sort_change(df_house_hold)

In [22]:
# NaN 값을 공백으로 변환 (Table에 저장하기 위함)
df_house_hold['리명'] = df_house_hold['리명'].fillna('')

In [23]:
df_house_hold.head()

,통계년월,법정동코드,시도명,시군구명,법정동명,리명,행정기관코드,행정동명,통,반,총인구수,세대수,세대당인구,남자인구수,여자인구수,남여비율,업데이트일자
0,202307,2720010100,대구광역시,남구,이천동,,2720051500,이천동,1,1,11,9,1.22,5,6,0.83,20230904
1,202307,2720010100,대구광역시,남구,이천동,,2720051500,이천동,1,2,47,25,1.88,23,24,0.96,20230904
2,202307,2720010100,대구광역시,남구,이천동,,2720051500,이천동,1,4,55,35,1.57,29,26,1.12,20230904
3,202307,2720010100,대구광역시,남구,이천동,,2720051500,이천동,1,5,71,52,1.37,36,35,1.03,20230904
4,202307,2720010100,대구광역시,남구,이천동,,2720051500,이천동,2,1,130,55,2.36,61,69,0.88,20230904


# 5. 법정동, 행정동 세대수 데이터  SQL에 삽입 

In [24]:
conn = dbconnect.db_connect("DEMO_DW")
cursor = conn.cursor()

query = f'SELECT EXISTS (SELECT * FROM household_dong WHERE 통계년월 = {month})'

cursor.execute(query)
row = cursor.fetchone()
data_exist = row[0]     # 저장된 데이터의 유무(1 - 데이터 있음)

conn.close()

In [25]:
if data_exist == 1:      # 데이터 존재하면
    print(f'{month} 에 데이터가 존재합니다.')
else:
    conn = dbconnect.db_connect("DEMO_DW")
    cur = conn.cursor()

    for row in df_house_hold.itertuples():
        sql = "insert into household_dong (통계년월, 법정동코드, 시도명, 시군구명, 법정동명, 리명, \
                                           행정기관코드, 행정동명, 통, 반, 총인구수, 세대수, 세대당인구, \
                                           남자인구수, 여자인구수, 남여비율, 업데이트일자) \
               values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        cur.execute(sql, (row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[9], row[10], row[11], row[12], row[13], row[14], row[15], row[16], row[17]))

    conn.commit()
    print(f'{month} 데이터를 저장하였습니다.')
    conn.close()

202307 데이터를 저장하였습니다.
